# Langchain Basics

## Initial Settings
### Load Libraries

In [1]:
import os
import openai
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import create_csv_agent

import pandas as pd
from dotenv import load_dotenv

### Set Environment Variables for Azure OpenAI Access

In [2]:
load_dotenv()
API_KEY = os.getenv("apikey")
BASE_ENDPOINT = os.getenv("api_base")
COMPLETION_MODEL = os.getenv("completion_model")
CHAT_MODEL = os.getenv("chat_model")
EMBEDDING_MODEL = os.getenv("embedding_model")
DEPLOYMENT_VERSION = os.getenv("api_version")

#init Azure OpenAI
openai.api_type = "azure"
openai.api_version = DEPLOYMENT_VERSION
openai.api_base = BASE_ENDPOINT
openai.api_key = API_KEY

### Initalizing Azure OpenAI Model Object

In [3]:
# Initializing Davinci Model for text completion 
llm = AzureOpenAI(
    deployment_name=COMPLETION_MODEL ,
    openai_api_key= API_KEY ,openai_api_version =DEPLOYMENT_VERSION
    )

In [4]:
# Use this format for Chatgpt or GPT4
gptchat = AzureChatOpenAI(
    deployment_name=CHAT_MODEL,
    openai_api_base = BASE_ENDPOINT,
    openai_api_key= API_KEY,
    openai_api_version="2023-03-15-preview"
)

### Simple Prompt Test

In [5]:
joke = llm("Tell me a dad joke")
print(joke)



Q: Why don't scientists trust atoms?
A: Because they make up everything!


### Simple Chat Test



In [6]:
response = gptchat(
    [
        SystemMessage(content="You are an AI assistant that helps a user with marketing content. Keep responses short and under 5 sentences."),
        HumanMessage(content="What are top movie trends in US")
    ]
)
response

AIMessage(content='As an AI, I cannot give real-time data, but the top movie trends in the US usually include superhero films, remakes or reboots, animated movies, adaptations of bestselling books, and streaming platform originals. Be sure to check the latest box office charts and streaming platforms for current trends.', additional_kwargs={}, example=False)

### Embeddings

In [7]:
openai_embed = OpenAIEmbeddings(model=EMBEDDING_MODEL,
                                openai_api_key=API_KEY)

query_embeddings = openai_embed.embed_query("The sky is blue") 

print(type(query_embeddings))
print(len(query_embeddings))
print(query_embeddings[:5])

<class 'list'>
1536
[0.009326309897005558, -0.0006310197059065104, 0.006163125857710838, -0.015400114469230175, -0.013527460396289825]


### Using Prompt Templates for use input and capturing any variables / non static informations

In [8]:
# Define a prompt template with placeholders for input variables.
prompt_template = PromptTemplate(
        template = 'Write a small advertisement a new {product} and its description {product_desc}?',
        input_variables = ['product', 'product_desc']
)

# Create the final prompt by filling in the defined variables.
final_prompt = prompt_template.format(product='Headphones',  product_desc='Best headphones')
print("FINAL PROMPT: " + final_prompt)

response = llm(final_prompt) 
print(response)

FINAL PROMPT: Write a small advertisement a new Headphones and its description Best headphones?
 Look no further!

Introducing the latest and greatest in sound technology: the G-Power Headphones. With superior sound quality and a comfortable fit, these headphones are the perfect addition to any music lover's collection. Featuring noise-cancelling technology and a long-lasting battery life, the G-Power Headphones deliver exceptional sound for an unbeatable price. Enjoy your favorite tunes with crystal clear clarity and deep bass. Get the G-Power Headphones today and experience the ultimate in audio quality.


In [9]:
# Reuse the prompt Template
print(llm(prompt_template.format(product ='Electric Vehicle', product_desc='Iceland')))



Introducing the new Electric Vehicle from Iceland – the perfect choice for your next green adventure! This vehicle has a range of up to 200 miles on a single charge, and a top speed of 80 mph. It is designed to handle the rugged terrain of Iceland's roads with ease, and is equipped with advanced safety features to help keep you and your passengers safe. Plus, the Electric Vehicle from Iceland helps reduce your carbon footprint and is eligible for various tax credits and incentives. Get ready to explore Iceland in style – with the Electric Vehicle from Iceland!


### Structured Output Parser

In [13]:
# 1. Define the schema

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="product", description="This is a product"),
    ResponseSchema(name="product_desc", description="This is a short description of the product")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [14]:
# 2. Generates output format instructions which can be included in your prompt.

format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"product": string  // This is a product
	"product_desc": string  // This is a short description of the product
}
```


In [15]:
# 3. Incorporate into final prompt.

template = '''
You will be given a product by the user.
Please provide a imaginary, creative and non offensive product name and a short description of the product.

{format_instructions}

% PRODUCT:
{product}

YOUR RESPONSE:
'''

prompt_template = PromptTemplate(
    input_variables=["product"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

prompt = prompt_template.format(product="Iphone")
print('FINAL PROMPT:')
print(prompt)

FINAL PROMPT:

You will be given a product by the user.
Please provide a imaginary, creative and non offensive product name and a short description of the product.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"product": string  // This is a product
	"product_desc": string  // This is a short description of the product
}
```

% PRODUCT:
Iphone

YOUR RESPONSE:



In [16]:
# 4. LLM now outputs an easily parsable response.

response = llm(prompt)
print(response)

```json
{
	"product": "Iphone",
	"product_desc": "The ultimate smartphone for the modern user, with an intuitive interface and a sleek design."
}
```


In [17]:
# 5. Output parser can extract named fields in the schema.

output_parser.parse(response)['product_desc']

'The ultimate smartphone for the modern user, with an intuitive interface and a sleek design.'

### Sequential Chains

In [18]:
# Chain component 1: 

from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

template1 = """You will be given an error from the user. Please provide a possible solution.

% ERROR
{error}

YOUR RESPONSE:
"""
prompt_template1 = PromptTemplate(input_variables=["error"], template=template1)

# LLMChain is a simple chain which takes a prompt template, formats with user input and returns LLM output.
error_chain = LLMChain(llm=llm, prompt=prompt_template1)

In [19]:
# Chain component 2: 

template2 = """Given the error message, provide a short solution for fixing it.

% MESSAGE
{message}

YOUR RESPONSE:
"""
prompt_template2 = PromptTemplate(input_variables=["message"], template=template2)

message_chain = LLMChain(llm=llm, prompt=prompt_template2)

In [20]:
# Run the chain
overall_chain = SimpleSequentialChain(
    chains=[error_chain, message_chain],
    verbose=True)
chain_output = overall_chain.run("ZeroDivisionError")



> Entering new  chain...
This error occurs when you attempt to divide a number by zero. To fix it, make sure that you are not dividing by zero. If you need to, you can use an if statement to check if the denominator is equal to zero before attempting the division.
To fix this error, make sure to check if the denominator is equal to zero before attempting the division. You can use an if statement to check if the denominator is equal to zero and then take the appropriate action if it is.

> Finished chain.


In [21]:
print(chain_output)

To fix this error, make sure to check if the denominator is equal to zero before attempting the division. You can use an if statement to check if the denominator is equal to zero and then take the appropriate action if it is.


### Document loader & text splitting**

In [22]:
from langchain.document_loaders import GutenbergLoader

# Load a long document.
loader = GutenbergLoader('https://www.gutenberg.org/cache/epub/1515/pg1515.txt')
doc = loader.load()

print(doc[0].page_content[:80].rstrip())
print('Document length in characters:', len(doc[0].page_content))

The Project Gutenberg eBook of The Merchant of Venice, by William Shakespeare
Document length in characters: 154162


In [23]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# We need to split long documents into smaller chunks to stay under the LLM token limit.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(doc[0].page_content)
texts[2]

'THE PRINCE OF MOROCCO, suitor to Portia\r\n\n\nTHE PRINCE OF ARRAGON, suitor to Portia\r\n\n\nANTONIO, a merchant of Venice\r\n\n\nBASSANIO, his friend, suitor to Portia\r\n\n\nGRATIANO, friend to Antonio and Bassanio\r\n\n\nSOLANIO, friend to Antonio and Bassanio\r\n\n\nSALARINO, friend to Antonio and Bassanio\r\n\n\nLORENZO, in love with Jessica\r\n\n\nSHYLOCK, a rich Jew\r\n\n\nTUBAL, a Jew, his friend\r\n\n\nLAUNCELET GOBBO, a clown, servant to Shylock\r\n\n\nOLD GOBBO, father to Launcelet\r\n\n\nLEONARDO, servant to Bassanio\r\n\n\nBALTHAZAR, servant to Portia\r\n\n\nSTEPHANO, servant to Portia\r\n\n\nSALERIO, a messenger from Venice\r\n\n\n\r\n\n\nPORTIA, a rich heiress\r\n\n\nNERISSA, her waiting-woman\r\n\n\nJESSICA, daughter to Shylock\r\n\n\n\r\n\n\nMagnificoes of Venice, Officers of the Court of Justice, a Gaoler,\r\n\n\nServants and other Attendants\r\n\n\n\r\n\n\nSCENE: Partly at Venice, and partly at Belmont, the seat of Portia on\r\n\n\nthe Continent\r\n\n\n\r\n\n\n\r\n

### Agents
#### CSV Agent Example

In [30]:
agent = create_csv_agent(llm, 'goalies17.csv', verbose=True)

In [32]:
agent.run("How many more shots did player Martin saved than adam?")



> Entering new  chain...
Thought: I need to calculate the difference between the two players' saved shots
Action: python_repl_ast
Action Input: df[df['Player'] == 'Martin Houle']['Saves'].values[0] - df[df['Player'] == 'Adam']['Saves'].values[0]
Observation: IndexError: index 0 is out of bounds for axis 0 with size 0
Thought: I need to find the player Adam
Action: python_repl_ast
Action Input: df[df['Player'].str.contains('Adam')]
Observation:             Player  First  Last  Active  Games Played  Games Started  Win  \
11     Adam Hauser   2005  2006       1             1            NaN  0.0   
183  Adam Berkhoel   2005  2006       1             9            NaN  2.0   
209     Adam Munro   2003  2006       2            17            NaN  4.0   
681     John Adams   1972  1975       2            22            NaN  9.0   

     Loss  Ties and Overtime Losses  Goals Against  Shot Saved  Saves  \
11    0.0                       0.0              6        24.0   18.0   
183   4.0         

'Martin Houle saved 16 more shots than Adam Hauser.'

### Prompt Template + Coversation Chain and  Memory 

In [5]:
# Create a converation chain
conversation = ConversationChain(
    llm=gptchat,
    memory=ConversationBufferMemory()
)

conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")
# -> The first three colors of a rainbow are red, orange, and yellow.
conversation.run("And the next 4?")
# -> The next four colors of a rainbow are green, blue, indigo, and violet.

'Green, blue, indigo, violet.'

In [7]:
template = """You are a article generation tool that generates articles for\
            car Brand. Use title similar to Number of Tips from Pros to Make Your vehicle perform the best.\
            The content should use details keywords and explain and extend them and use {content}. \
            The word count of the article should be minimum 1000 words
            {chat_history}
            Human: {content}
            Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "content"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [8]:
llm_chain = LLMChain(
    llm=gptchat,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [9]:
intro_text = llm_chain.predict(content= "Write an introductory paragraph that gives context to the reader, introducing the overall theme of the article. Try to include the target keywords")



> Entering new  chain...
Prompt after formatting:
You are a article generation tool that generates articles for            car Brand. Use title similar to Number of Tips from Pros to Make Your vehicle perform the best.            The content should use details keywords and explain and extend them and use Write an introductory paragraph that gives context to the reader, introducing the overall theme of the article. Try to include the target keywords.             The word count of the article should be minimum 1000 words
            
            Human: Write an introductory paragraph that gives context to the reader, introducing the overall theme of the article. Try to include the target keywords
            Chatbot:

> Finished chain.


In [10]:
intro_text

"In today's fast-paced world, car enthusiasts and everyday drivers alike are constantly seeking ways to optimize their vehicle's performance. This article is your ultimate guide to unlocking your car's full potential, as we bring you a comprehensive list of tips from the pros to make your vehicle perform at its best. Whether you're driving a luxury car or a budget-friendly option, our expert advice and detailed keywords will steer you in the right direction to achieve peak performance on the road. Get ready to rev up your engine and embark on a journey towards enhancing your driving experience with our top-notch recommendations."

In [11]:
content_1 = llm_chain.predict(content= "Discuss how car should be kept clean, well serviced and well maintained")



> Entering new  chain...
Prompt after formatting:
You are a article generation tool that generates articles for            car Brand. Use title similar to Number of Tips from Pros to Make Your vehicle perform the best.            The content should use details keywords and explain and extend them and use Discuss how car should be kept clean, well serviced and well maintained.             The word count of the article should be minimum 1000 words
            Human: Write an introductory paragraph that gives context to the reader, introducing the overall theme of the article. Try to include the target keywords
AI: In today's fast-paced world, car enthusiasts and everyday drivers alike are constantly seeking ways to optimize their vehicle's performance. This article is your ultimate guide to unlocking your car's full potential, as we bring you a comprehensive list of tips from the pros to make your vehicle perform at its best. Whether you're driving a luxury car or a budget-friendly opt

In [12]:
content_1

"A clean, well-serviced, and well-maintained car is not only aesthetically pleasing but is also crucial for optimal performance, safety, and longevity. Here are some key aspects you should focus on to ensure your vehicle stays in top condition:\n\n1. Regular car washes: Keeping your car clean not only enhances its appearance but also prevents dirt, grime, and other contaminants from causing damage to the paintwork and other components. Washing your car regularly, using a high-quality car shampoo and microfiber cloths, can help maintain its shine and protect the finish.\n\n2. Waxing and polishing: Applying a quality wax or polish can create a protective barrier on your car's paintwork, shielding it from the harsh effects of the sun, rain, and other environmental factors. Waxing and polishing also help keep your car looking new and glossy, while minimizing the risk of scratches and swirl marks.\n\n3. Interior cleaning: A clean interior not only contributes to a more enjoyable driving exp